Add necessary imports here

In [1]:
import nltk, string, os, json

from nltk import RegexpParser
from nltk.tree import Tree

from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Add necessary dictionaries/lists and parsing methods here

In [0]:
def get_article(filename):
    with open(filename, "r") as f:
        s = [] # steps - list of tuples (method_number, step text)
        d = {} # descriptions - Description dictionary step text -> description
        m = [] # methods
        data_json = json.load(f)
        title = data_json['title']
        if title != None:
        # Get the steps
            if len(data_json['method/part']) > 0:
                methods = data_json['method/part']
                for method in methods:
                    all_steps = [(method['name'], step[0]) for step in method['steps']]
                    s += all_steps
                    m += method['name']
                    all_descs = dict(zip([step[0] for step in method['steps']], [step[1] for step in method['steps']]))
                    d.update(all_descs)      
            else:
                s = [(None, step[0]) for step in data_json['steps']]
                d = dict(zip([step[0] for step in data_json['steps']], [step[1] for step in data_json['steps']]))
        else:
            print('Article has no title!')
        return s, d, m, title

Add NLTK POS-tagging and tokenizer here

In [18]:
nltk.download() # download averaged_perceptron_tagger

# Source: https://stackoverflow.com/questions/29473169/approach-for-identifying-whether-a-sentence-includes-an-imperative-within-it?fbclid=IwAR1kGWHvo0M1sSMMXl4Rv4PJ-dhRchEABc83JQpAqotQnFqY3uUBD0jAMAU

def is_imperative(tagged_sent):
    # if the sentence is not a question...
    if tagged_sent[-1][0] != "?":
         #catches simple imperatives, e.g. "Open the pod bay doors, HAL!"
        
        if tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD":
            return True
        
        # catches imperative sentences starting with words like 'please', 'you',...
        # E.g. "Dave, stop.", "Just take a stress pill and think things over."
        else:
            chunk = get_chunks(tagged_sent)
            # check if the first chunk of the sentence is a VB-Phrase
            if type(chunk[0]) is Tree and chunk[0].label() == "VB-Phrase":
                return True

    # Questions can be imperatives too, let's check if this one is
    else:
        # check if sentence contains the word 'please'
        pls = len([w for w in tagged_sent if w[0].lower() == "please"]) > 0
        # catches requests disguised as questions
        # e.g. "Open the doors, HAL, please?"
        chunk = get_chunks(tagged_sent)
        if pls and (tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD"):
            return True

        # catches imperatives ending with a Question tag
        # and starting with a verb in base form, e.g. "Stop it, will you?"
        elif type(chunk[-1]) is Tree and chunk[-1].label() == "Q-Tag":
            if (chunk[0][1] == "VB" or
                (type(chunk[0]) is Tree and chunk[0].label() == "VB-Phrase")):
                return True

    return False

# chunks the sentence into grammatical phrases based on its POS-tags
def get_chunks(tagged_sent):
    chunkgram = r"""VB-Phrase: {<DT><,>*<VB>}
                    VB-Phrase: {<RB><VB>}
                    VB-Phrase: {<UH><,>*<VB>}
                    VB-Phrase: {<UH><,><VBP>}
                    VB-Phrase: {<PRP><VB>}
                    VB-Phrase: {<NN.?>+<,>*<VB>}
                    Q-Tag: {<,><MD><RB>*<PRP><.>*}"""
    chunkparser = RegexpParser(chunkgram)
    return chunkparser.parse(tagged_sent)

# Tokenizer that I implemented from CIS421
def tokenize(text):
    tokens = []
    words = text.split(' ')
    for word in words:
        if len(word) == 0:
            continue
        w = []
        for c in word:
            if c in string.punctuation:
                if len(w) != 0:
                    tokens.append(''.join(w))
                    w = []
                tokens.append(c)
            else:
                if c not in string.whitespace:
                    w.append(c)
        if len(w) != 0:
            tokens.append(''.join(w))
    return tokens

def imperative_check(text):
  # First tokenize the step headline
  tokens = tokenize(text)
  # Get the POS-tags
  tags = nltk.pos_tag(tokens)
  # Check if imperative
  imperative = is_imperative(tags)
  
  return imperative

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> averaged_perceptron_tagger
        /root/nltk_data...
      Unzipping taggers/averaged_perceptron_tagger.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


Add similarity check functions here

In [10]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

     |████████████████████████████████| 5.4MB 3.5MB/s 
  Created wheel for pymagnitude: filename=pymagnitude-0.1.120-cp36-cp36m-linux_x86_64.whl size=135918206 sha256=b57b46aad0016f367e5dfde76fc41ac687bf0a79eb852334f49b502b32d55705
  Stored in directory: /root/.cache/pip/wheels/a2/c7/98/cb48b9db35f8d1a7827b764dc36c5515179dc116448a47c8a1
Successfully built pymagnitude
--2020-04-09 01:46:28--  http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.216.168.194
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.216.168.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1384890368 (1.3G) [binary/octet-stream]
Saving to: ‘glove.6B.300d.magnitude’

glove.6B.300d.magni 100%[===================>]   1.29G  46.5MB/s    in 30s     

2020-04-09 01:46:59 (43.4 MB/s) - ‘glove.6B.300d.magnitude’ saved [1384890368/1384890368]



In [0]:
#get vector for the given sentence
def construct_sentence_vector(command, vectors):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  for word in command.split():
    word_vector = vectors.query(word)
    sentence_vector += word_vector
  sentence_vector = sentence_vector / len(command.split())
  return sentence_vector

#compare items in the list to the article_field. for instance, if you want to find the most similar titles
#article_field should be the title, and list should be a list of titles you want to find the most similar out of
def find_most_similar(article_field, list, vectors):
  vecs = {}
  max_sim = 0
  most_sim = list[0]
  vector_1 = construct_sentence_vector(article_field, vectors)
  for obj in list:
    vecs[obj] = construct_sentence_vector(obj, vectors)
  for obj in vecs:
    sim = vectors.similarity(vecs[obj], vector_1)
    if sim > max_sim:
      most_sim = obj
      max_sim = sim
  return most_sim

#get the top k most similar
def top_k_similar(article_field, list, vectors, k):
  top = []
  for i in range(k):
    most_sim = find_most_similar(article_field, list, vectors)
    top.append(most_sim)
    list.remove(most_sim)
  return top


Add object identifier & 3rd person sentence formation here

In [0]:
# refered from https://stackoverflow.com/questions/41051125/turning-a-sentence-from-first-to-second-person

forms = {"am" : "are", "are" : "am", 'i' : 'you', 'my' : 'your', 'me' : 'you', 'mine' : 'yours', 'you' : 'I', 
'your' : 'my', 'yours' : 'mine'}

def translate(text):
  # print(text)
  result = text
  for word in text.split():
    # print(word)
    if word.lower() in forms: 
      result = result.replace(word, forms[word.lower()])
      # print(result)
  # print(result)
  return result

def changePerson(text):
  result = ""
  tags = nltk.pos_tag(tokenize(text))
  isImp = is_imperative(tags)

  if isImp:
    if text[0] == ' ' or text[0] == '\n':
      result = "You " + text[1].lower() + text[2:]
    else:
      result = "You " + text[0].lower() + text[1:]
  else:
    result = translate(text)
  return result

def identifyObjects(text):
  tags = nltk.pos_tag(tokenize(text))
  objects = []
  for t in tags:
    if t[1]=="NN":
      objects.append(t[0])
  return objects

Once imports and functions are done, upload a JSON file, get its name and parse it

In [2]:
uploaded = files.upload()

Saving 1766.json to 1766.json
{'1766.json': b'{\n    "title": "How to Travel by Air",\n    "url": "https://www.wikihow.com/Travel-by-Air",\n    "title_description": "Whether you\'re taking a long-awaited vacation or meeting with potential clients, traveling by air can sometimes be quite a hassle. This is especially true in the post-9/11 world. By making just a few sensible preparations before you board that plane, you can be less stressed and more comfortable. In fact, you can even make things easier on your fellow travelers!",\n    "category_hierarchy": [\n        "s & Other Vehicles",\n        "Aviation"\n    ],\n    "method/part": [\n        {\n            "name": "Packing the Essentials",\n            "steps": [\n                [\n                    "Pack what you need",\n                    " Just like for any holiday, only pack the things you need. A crucial thing you should do before flying is to check the airline\'s baggage allowances so you get a rough idea of how much stuff

In [53]:
file_name = list(uploaded.keys())[0]

def milestone_1_results(filename):
  steps, descriptions, method_names, title = get_article(filename)

  # print the title of the article
  print('Title: ' + title)
  # Once the necessary fields are filled, find imperatives
  imperatives = []

  for step in steps:
    # First split the headline into sentences
    headline_sentences = step[1].split('.')
    for headline in headline_sentences:
      if len(headline) == 0:
          continue
      if imperative_check(headline):
        if headline[0] == ' ':
          imperatives.append(headline[1:])
        else:
          imperatives.append(headline)
        break
    description = descriptions[step[1]]
    # Split the description paragraph into sentences
    sentences = description.split('.')
    for sentence in sentences:
      if len(sentence) == 0:
        continue
      if imperative_check(sentence):
        if sentence[0] == ' ':
          imperatives.append(sentence[1:])
        else:
          imperatives.append(sentence)
    
  # Once imperatives are found, print them and turn them into 3rd person
  print('\nList of Imperatives:\n')
  for imp in imperatives:
    print(imp)
    
  third_person = [changePerson(imperative) for imperative in imperatives]

  # Print the 3rd person sentences in order
  print('\nImperatives Turned Into 3rd Person:\n')
  for third in third_person:
    print(third)
  
  # Find 5 similar articles to this article given the title
  k = 5

  # Make sure that the folder new_wikihow_1000 is in the drive - that's where we have all the data
  file_path = '/content/drive/My Drive/Colab Notebooks/new_wikihow_1000'
  all_files = os.listdir(file_path)
  files_and_titles = {} # title -> filename dictionary
  idx = 0
  for data in all_files:
    if idx >= 100: # for now the number of files to read is limited to 100
      break
    with open(file_path + '/' + data, "r") as f:
      data_json = json.load(f)
      t = data_json['title']
      # Skip articles with no title
      if t == None:
        continue
      files_and_titles[t] = data
      idx += 1
  
  # Once we have file titles, pass them as a list to top_k_similar()
  k_similar = top_k_similar(title, list(files_and_titles.keys()), vectors, k)
  
  # Print the top k similar article titles and the corresponding json file name
  print('\nTop ' + str(k) + ' Similar Article Titles:\n')
  for similar in k_similar:
    print('Title: ' + similar + ', Filename: ' + files_and_titles[similar])

milestone_1_results(file_name)


Title: How to Travel by Air

List of Imperatives:

Pack what you need
Bring your visa (If needed), passport, tickets, and other documents you may need when you are on this trip
Be sure to bring these in your carry on or pockets—not in your checked luggage! The best thing to do would be to bring your important travel documents in pockets with zips, as it reduces the chances of the documents falling out of the pockets and you having a very hard time explaining to security
Keep one copy of your documents with you but separate from the original
Leave another copy to a trusted friend at home
Try to check in online or on your phone, as this means that you have the chance to make multiple copies of boarding passes as backups and makes it a lot easier, quicker, and a lot less stressful when you get to the actual airport
Should it become necessary to evacuate the plane by using the inflatable slide, or worse, walking out onto the wing, the last things you want to be wearing are shorts, a tank t